In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# Get the data from the model

In [2]:
y_pred = pd.read_pickle('../data/pkl/y_pred_60.pkl')
y_pred['left_0.6'] = round(y_pred['left_0.6'], 1)
y_pred['right_0.6'] = round(y_pred['right_0.6'], 1)
y_pred

,GAME_ID,GAME_DATE,TEAM_NAME_h,TEAM_NAME_a,left_0.6,right_0.6
0,22200973,2023-03-05,Wizards,Bucks,-5.3,-9.0
1,22200970,2023-03-05,Clippers,Grizzlies,-2.3,-7.1
2,22200969,2023-03-05,Celtics,Knicks,7.1,4.2
3,22200968,2023-03-05,Thunder,Jazz,-0.4,-3.7
4,22200967,2023-03-05,Rockets,Spurs,-2.0,-4.7
...,...,...,...,...,...,...
1116,42100231,2022-05-01,Grizzlies,Warriors,5.0,1.6
1117,42100211,2022-05-01,Celtics,Bucks,3.2,0.2
1118,42100156,2022-04-29,Timberwolves,Grizzlies,-1.0,-5.4
1119,42100176,2022-04-28,Jazz,Mavericks,-1.7,-5.0


In [3]:
# y_pred = pd.read_pickle('../data/pkl/y_pred_adv_test.pkl')
# y_pred = y_pred[['GAME_ID', 'GAME_DATE', 'TEAM_NAME_h', 'TEAM_NAME_a', 'y_pred', 'PLUS_MINUS']]
# y_pred.drop_duplicates(subset='GAME_ID', inplace=True)
# y_pred['y_pred'] = round(y_pred['y_pred'], 0)
# y_pred.sort_values('GAME_DATE', ascending=False)

# Get the historical betting data

In [4]:
bets = pd.read_pickle('../data/pkl/sbr_betting_data.pkl')
bets = bets.reset_index(drop=True)
bets['Game_Date'] = pd.to_datetime(bets['Game_Date'])

In [5]:
bets

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm_Spread,Betmgm_Odds,...,Fanduel_Spread,Fanduel_Odds,Caesars_Spread,Caesars_Odds,Pointsbet_Spread,Pointsbet_Odds,Wynn_Spread,Wynn_Odds,Betrivers_Spread,Betrivers_Odds
0,2023-03-05,Phoenix,0,Dallas,130,56%,+1.5,-110,-1.5,-115,...,-2,-108,-2,-110,-2,-110,-2,-110,-2,-112
1,2023-03-05,Dallas,1,Phoenix,126,44%,-1.5,-110,+1.5,-105,...,+2,-112,+2,-110,+2,-110,+2,-110,+2,-112
2,2023-03-05,Indiana,0,Chicago,125,53%,+6.5,-110,+5.5,-110,...,+5.5,-108,+5,-110,+5.5,-110,+5.5,-110,+5.5,-110
3,2023-03-05,Chicago,1,Indiana,122,47%,-6.5,-110,-5.5,-110,...,-5.5,-112,-5,-110,-5.5,-110,-5.5,-110,-5.5,-112
4,2023-03-05,Golden State,0,L.A. Lakers,105,60%,-5.5,-110,-5.5,-105,...,-5.5,-118,-5.5,-110,-5,-110,-5,-110,-5,-109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,2019-10-05,San Antonio,1,Orlando,89,-,-5.5,-110,,-,...,,-,,-,-6.5,-110,,-,,-
9862,2019-10-04,Houston,0,L.A. Clippers,109,-,-4.5,-110,,-,...,,-,,-,-5,-110,,-,,-
9863,2019-10-04,L.A. Clippers,1,Houston,96,-,+4.5,-110,,-,...,,-,,-,+5,-110,,-,,-
9864,2019-10-04,Indiana,0,Sacramento,132,-,+2.5,-110,,-,...,,-,,-,+5.5,-110,,-,,-


## Fix the messed up column names from the pickle data

In [6]:
# temp = ['Draft_Kings', 'Fanduel', 'Caesars', 'Pointsbet', 'Wynn', 'Betrivers']
# merge_df = bets[['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets',
#        'Opening_Spread', 'Opening_Odds', 'Betmgm', 'Betmgm_Odds']]
# for i in range(12):
#     if i % 2 == 0:
#         name = temp[int(i/2)]
#         temp_df = pd.DataFrame(bets.iloc[:, (i+10)]).rename(columns={f'{name}_Odds': f'{name}_Spread'}).reset_index(drop=True)
#     else:
#         name = temp[int(i/2)-1]
#         temp_df = pd.DataFrame(bets.iloc[:, (i+10)]).rename(columns={f'{name}_Odds': f'{name}_Odds'}).reset_index(drop=True)
#     merge_df = merge_df.merge(temp_df, left_index=True, right_index=True)
# bets = merge_df.copy()
# bets.rename(columns={'Betmgm': 'Betmgm_Spread'}, inplace=True)

In [7]:
bets.columns

Index(['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets',
       'Opening_Spread', 'Opening_Odds', 'Betmgm_Spread', 'Betmgm_Odds',
       'Draft_Kings_Spread', 'Draft_Kings_Odds', 'Fanduel_Spread',
       'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds', 'Pointsbet_Spread',
       'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds', 'Betrivers_Spread',
       'Betrivers_Odds'],
      dtype='object')

In [8]:
sportsbooks_spreads = ['Opening_Spread','Betmgm_Spread', 'Draft_Kings_Spread', 'Fanduel_Spread', 
                       'Caesars_Spread', 'Pointsbet_Spread', 'Wynn_Spread', 'Betrivers_Spread']

sportsbooks_odds = ['Opening_Odds', 'Betmgm_Odds', 'Draft_Kings_Odds', 'Fanduel_Odds', 
                    'Caesars_Odds', 'Pointsbet_Odds', 'Wynn_Odds', 'Betrivers_Odds']

sportsbooks_spreads_odds = ['Betmgm_Spread', 'Betmgm_Odds',
       'Draft_Kings_Spread', 'Draft_Kings_Odds', 'Fanduel_Spread',
       'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds', 'Pointsbet_Spread',
       'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds', 'Betrivers_Spread',
       'Betrivers_Odds']

### Replace missing values with nan

In [9]:
bets = bets.replace(to_replace=['-', ''], value='1000')
bets = bets.replace(to_replace=['PK'], value='0')

In [10]:
bets[sportsbooks_spreads] = bets[sportsbooks_spreads].astype('float32')
bets[sportsbooks_odds] = bets[sportsbooks_odds].astype('int32')

In [11]:
bets = bets.replace(to_replace=[1000, '1000'], value=[np.nan, ''])

# Compare the data

In [12]:
y_pred['TEAM_NAME_a'].sort_values().unique()
team_name = ['Hawks', 'Celtics', 'Nets', 'Hornets', 'Bulls', 'Cavaliers', 'Mavericks', 'Nuggets', 'Pistons',
            'Warriors', 'Rockets', 'Pacers', 'Clippers', 'Lakers', 'Grizzlies', 'Heat', 'Bucks', 'Timberwolves',
            'Pelicans', 'Knicks', 'Thunder', 'Magic', '76ers', 'Suns', 'Trail Blazers', 'Kings', 'Spurs',
            'Raptors', 'Jazz', 'Wizards']
team_city = bets['Team_Name'].sort_values().unique().tolist()
map = {}
for i, name in enumerate(team_name):
    map[name] = team_city[i]
y_pred = y_pred.replace({'TEAM_NAME_h': map, 'TEAM_NAME_a': map})

In [13]:
def home_team(row):
    return [row.Opponent if row.Home==0 else row.Team_Name][0]
home=[]
for i in range(len(bets)):
    row=bets.iloc[i]
    home.append([row.Opponent if row.Home==0 else row.Team_Name][0])
bets['TEAM_NAME_h'] = home
bets

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm_Spread,Betmgm_Odds,...,Fanduel_Odds,Caesars_Spread,Caesars_Odds,Pointsbet_Spread,Pointsbet_Odds,Wynn_Spread,Wynn_Odds,Betrivers_Spread,Betrivers_Odds,TEAM_NAME_h
0,2023-03-05,Phoenix,0,Dallas,130,56%,1.5,-110,-1.5,-115.0,...,-108.0,-2.0,-110.0,-2.0,-110.0,-2.0,-110.0,-2.0,-112.0,Dallas
1,2023-03-05,Dallas,1,Phoenix,126,44%,-1.5,-110,1.5,-105.0,...,-112.0,2.0,-110.0,2.0,-110.0,2.0,-110.0,2.0,-112.0,Dallas
2,2023-03-05,Indiana,0,Chicago,125,53%,6.5,-110,5.5,-110.0,...,-108.0,5.0,-110.0,5.5,-110.0,5.5,-110.0,5.5,-110.0,Chicago
3,2023-03-05,Chicago,1,Indiana,122,47%,-6.5,-110,-5.5,-110.0,...,-112.0,-5.0,-110.0,-5.5,-110.0,-5.5,-110.0,-5.5,-112.0,Chicago
4,2023-03-05,Golden State,0,L.A. Lakers,105,60%,-5.5,-110,-5.5,-105.0,...,-118.0,-5.5,-110.0,-5.0,-110.0,-5.0,-110.0,-5.0,-109.0,L.A. Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,2019-10-05,San Antonio,1,Orlando,89,,-5.5,-110,NaN,NaN,...,NaN,NaN,NaN,-6.5,-110.0,NaN,NaN,NaN,NaN,San Antonio
9862,2019-10-04,Houston,0,L.A. Clippers,109,,-4.5,-110,NaN,NaN,...,NaN,NaN,NaN,-5.0,-110.0,NaN,NaN,NaN,NaN,L.A. Clippers
9863,2019-10-04,L.A. Clippers,1,Houston,96,,4.5,-110,NaN,NaN,...,NaN,NaN,NaN,5.0,-110.0,NaN,NaN,NaN,NaN,L.A. Clippers
9864,2019-10-04,Indiana,0,Sacramento,132,,2.5,-110,NaN,NaN,...,NaN,NaN,NaN,5.5,-110.0,NaN,NaN,NaN,NaN,Sacramento


In [14]:
bets['Game_Date'] = pd.to_datetime(bets['Game_Date'])

In [15]:
y_pred

,GAME_ID,GAME_DATE,TEAM_NAME_h,TEAM_NAME_a,left_0.6,right_0.6
0,22200973,2023-03-05,Washington,Milwaukee,-5.3,-9.0
1,22200970,2023-03-05,L.A. Clippers,Memphis,-2.3,-7.1
2,22200969,2023-03-05,Boston,New York,7.1,4.2
3,22200968,2023-03-05,Oklahoma City,Utah,-0.4,-3.7
4,22200967,2023-03-05,Houston,San Antonio,-2.0,-4.7
...,...,...,...,...,...,...
1116,42100231,2022-05-01,Memphis,Golden State,5.0,1.6
1117,42100211,2022-05-01,Boston,Milwaukee,3.2,0.2
1118,42100156,2022-04-29,Minnesota,Memphis,-1.0,-5.4
1119,42100176,2022-04-28,Utah,Dallas,-1.7,-5.0


In [16]:
compare = bets.merge(y_pred, left_on=['Game_Date', 'TEAM_NAME_h'], right_on=['GAME_DATE', 'TEAM_NAME_h'])

In [17]:
compare

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm_Spread,Betmgm_Odds,...,Wynn_Spread,Wynn_Odds,Betrivers_Spread,Betrivers_Odds,TEAM_NAME_h,GAME_ID,GAME_DATE,TEAM_NAME_a,left_0.6,right_0.6
0,2023-03-05,Phoenix,0,Dallas,130,56%,1.5,-110,-1.5,-115.0,...,-2.0,-110.0,-2.0,-112.0,Dallas,22200962,2023-03-05,Phoenix,5.6,2.2
1,2023-03-05,Dallas,1,Phoenix,126,44%,-1.5,-110,1.5,-105.0,...,2.0,-110.0,2.0,-112.0,Dallas,22200962,2023-03-05,Phoenix,5.6,2.2
2,2023-03-05,Indiana,0,Chicago,125,53%,6.5,-110,5.5,-110.0,...,5.5,-110.0,5.5,-110.0,Chicago,22200963,2023-03-05,Indiana,-2.1,-5.8
3,2023-03-05,Chicago,1,Indiana,122,47%,-6.5,-110,-5.5,-110.0,...,-5.5,-110.0,-5.5,-112.0,Chicago,22200963,2023-03-05,Indiana,-2.1,-5.8
4,2023-03-05,Golden State,0,L.A. Lakers,105,60%,-5.5,-110,-5.5,-105.0,...,-5.0,-110.0,-5.0,-109.0,L.A. Lakers,22200964,2023-03-05,Golden State,-1.1,-4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,2022-04-29,Minnesota,1,Memphis,106,50%,1.5,-110,1.5,-110.0,...,2.0,-110.0,2.0,-110.0,Minnesota,42100156,2022-04-29,Memphis,-1.0,-5.4
2140,2022-04-28,Philadelphia,0,Toronto,132,40%,-1.5,-110,-1.5,-105.0,...,0.0,-108.0,-1.0,-109.0,Toronto,42100136,2022-04-28,Philadelphia,-4.4,-8.2
2141,2022-04-28,Toronto,1,Philadelphia,97,60%,1.5,-110,1.5,-115.0,...,0.0,-108.0,1.0,-112.0,Toronto,42100136,2022-04-28,Philadelphia,-4.4,-8.2
2142,2022-04-28,Dallas,0,Utah,98,56%,1.5,-110,-1.5,-110.0,...,-1.5,-108.0,-1.0,-109.0,Utah,42100176,2022-04-28,Dallas,-1.7,-5.0


# Compare the data using selection

## To add later:

Change from 'Opening Spread' to the best available spread for that bet, also get the odd associated with that bet and calculate actual money value of bets

In [18]:
compare = compare[['Game_Date', 'Team_Name', 'Home', 'Opponent','Opening_Spread', 'left_0.6', 'right_0.6']]
compare

,Game_Date,Team_Name,Home,Opponent,Opening_Spread,left_0.6,right_0.6
0,2023-03-05,Phoenix,0,Dallas,1.5,5.6,2.2
1,2023-03-05,Dallas,1,Phoenix,-1.5,5.6,2.2
2,2023-03-05,Indiana,0,Chicago,6.5,-2.1,-5.8
3,2023-03-05,Chicago,1,Indiana,-6.5,-2.1,-5.8
4,2023-03-05,Golden State,0,L.A. Lakers,-5.5,-1.1,-4.6
...,...,...,...,...,...,...,...
2139,2022-04-29,Minnesota,1,Memphis,1.5,-1.0,-5.4
2140,2022-04-28,Philadelphia,0,Toronto,-1.5,-4.4,-8.2
2141,2022-04-28,Toronto,1,Philadelphia,1.5,-4.4,-8.2
2142,2022-04-28,Dallas,0,Utah,1.5,-1.7,-5.0


In [19]:
compare['y_pred'] = np.where((compare['Home'] == 1), compare['right_0.6'], compare['left_0.6'])

/tmp/ipykernel_4726/2543364173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare['y_pred'] = np.where((compare['Home'] == 1), compare['right_0.6'], compare['left_0.6'])


In [20]:
compare = compare.drop(columns=['left_0.6', 'right_0.6'])
compare

,Game_Date,Team_Name,Home,Opponent,Opening_Spread,y_pred
0,2023-03-05,Phoenix,0,Dallas,1.5,5.6
1,2023-03-05,Dallas,1,Phoenix,-1.5,2.2
2,2023-03-05,Indiana,0,Chicago,6.5,-2.1
3,2023-03-05,Chicago,1,Indiana,-6.5,-5.8
4,2023-03-05,Golden State,0,L.A. Lakers,-5.5,-1.1
...,...,...,...,...,...,...
2139,2022-04-29,Minnesota,1,Memphis,1.5,-5.4
2140,2022-04-28,Philadelphia,0,Toronto,-1.5,-4.4
2141,2022-04-28,Toronto,1,Philadelphia,1.5,-8.2
2142,2022-04-28,Dallas,0,Utah,1.5,-1.7


In [21]:
home = compare[compare['Home'] == 1]
away = compare[compare['Home'] == 0]

In [22]:
home['y_pred'] = home['y_pred'] * -1
# home['PLUS_MINUS'] = home['PLUS_MINUS'] * -1

/tmp/ipykernel_4726/1516273910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home['y_pred'] = home['y_pred'] * -1


In [23]:
compare = pd.concat([away, home]).sort_index()

In [24]:
# def bet_or_not(spread, pred):
#     if pred < spread:
#         return 'bet'
#     else:
#         return "don't bet"

In [25]:
compare['bet'] = np.where((compare['y_pred'] < compare['Opening_Spread']), True, False)
# compare['win'] = np.where((compare['PLUS_MINUS'] < compare['Opening_Spread']), True, False)

In [26]:
compare['bet'].value_counts()

False    1367
True      777
Name: bet, dtype: int64

In [27]:
compare

,Game_Date,Team_Name,Home,Opponent,Opening_Spread,y_pred,bet
0,2023-03-05,Phoenix,0,Dallas,1.5,5.6,False
1,2023-03-05,Dallas,1,Phoenix,-1.5,-2.2,True
2,2023-03-05,Indiana,0,Chicago,6.5,-2.1,True
3,2023-03-05,Chicago,1,Indiana,-6.5,5.8,False
4,2023-03-05,Golden State,0,L.A. Lakers,-5.5,-1.1,False
...,...,...,...,...,...,...,...
2139,2022-04-29,Minnesota,1,Memphis,1.5,5.4,False
2140,2022-04-28,Philadelphia,0,Toronto,-1.5,-4.4,True
2141,2022-04-28,Toronto,1,Philadelphia,1.5,8.2,False
2142,2022-04-28,Dallas,0,Utah,1.5,-1.7,True
